Modes of experimentation: Model (oracle, personal), Granularity (each recipe for all users, all recipes for all users), Surprise (95th percentile, max), Method of correlation (spearman, pearson)

Libraries


In [1]:
import os, pickle, numpy as np
from GloVex.evaluate_personalised import survey_reader
from scipy.stats import pearsonr, spearmanr
cwd = os.getcwd()

Get the oracle surprise estimates by the model

In [2]:
oracle_suprise_estimates_pickle_fn = cwd + '/GloVex/results/new/oracle_suprise_estimates.pickle'
oracle_suprise_estimates_dict = pickle.load(open(oracle_suprise_estimates_pickle_fn, 'rb'))

Get the oracle's surprise estimates (95th percentile and max score)

In [3]:
# Initialize arrays
oracle_surp_estimates_95perc = []
oracle_surp_estimates_max = []
# Iterate over the dict
for each_recipe in oracle_suprise_estimates_dict:
	# Get all of the ingredients combinations' surprise estimates
	oracle_ingr_surp_estimates = [each_ingr_comb[2] for each_ingr_comb in oracle_suprise_estimates_dict[each_recipe]['surprise_cuisine']]
	# Append the 95th_percentile surprise estimate
	oracle_surp_estimates_95perc.append(oracle_suprise_estimates_dict[each_recipe]['95th_percentile'])
	# Append the max score surprise estimate
	oracle_surp_estimates_max.append(max(oracle_ingr_surp_estimates))
# oracle_surp_estimates_95perc
# oracle_surp_estimates_max

Get the personal surprise estimates by the model

In [4]:
user_suprise_estimates_pickle_fn = cwd + '/GloVex/results/new/user_suprise_estimates.pickle'
user_suprise_estimates_dict = pickle.load(open(user_suprise_estimates_pickle_fn,'rb'))

Get the 95th percentils and max surprise estimates

In [5]:
# Initialize surprise estimates for 95th percentile and max score
personalized_surp_estimates_95perc = []
personalized_surp_estimates_max = []
# Iterate over the user surprise estimeates dict
for each_user in user_suprise_estimates_dict:
	# print 'each_user', each_user, user_suprise_estimates_dict[each_user]
	# Initialize the inner arrays for 95th percentile and max score
	users_surp_estimates_95perc = []
	users_surp_estimates_max = []
	# Iterate over the surprises for each user's surprise estimates
	for recipe_idx, each_recipe in enumerate(user_suprise_estimates_dict[each_user]['recipes_surp']):
		# print recipe_idx, user_suprise_estimates_dict[each_user]['recipes_surp'][recipe_idx]['95th_percentile']
		# Get all of the ingredients combinations' surprise estimates
		ingr_comb_suprise_arr = [each_ingr_comb[2] for each_ingr_comb in user_suprise_estimates_dict[each_user]['recipes_surp'][recipe_idx]['surprise_cuisine']]
		# print ingr_comb_suprise_arr
		# print max(ingr_comb_suprise_arr)
		# Append the 95th percentile for the surprise estimates
		users_surp_estimates_95perc.append(user_suprise_estimates_dict[each_user]['recipes_surp'][recipe_idx]['95th_percentile'])
		# Append the max score for the surprise estimates
		users_surp_estimates_max.append(max(ingr_comb_suprise_arr))
	# Append the user's surprise estimates to the personalized main 2D array (95th percentile and max score)
	personalized_surp_estimates_95perc.append(users_surp_estimates_95perc)
	personalized_surp_estimates_max.append(users_surp_estimates_max)
# personalized_surp_estimates_95perc
# personalized_surp_estimates_max


Transpose for recipe level comparison

In [6]:
personalized_surp_estimates_recipelvl_95perc = list(np.transpose(personalized_surp_estimates_95perc))
personalized_surp_estimates_recipelvl_max = list(np.transpose(personalized_surp_estimates_max))

Flatten the arrays in both dimnessions

In [8]:
personalized_surp_estimates_userflat_95perc = [item for sublist in personalized_surp_estimates_95perc for item in sublist]
personalized_surp_estimates_userflat_max = [item for sublist in personalized_surp_estimates_max for item in sublist]
personalized_surp_estimates_recipeflat_95perc = [item for sublist in personalized_surp_estimates_recipelvl_95perc for item in sublist]
personalized_surp_estimates_recipeflat_max = [item for sublist in personalized_surp_estimates_recipelvl_max for item in sublist]

Get the surprises of the users in the survey

In [9]:
survey_fp = '/Volumes/GoogleDrive/My Drive/Live/PQE/Omar/personalized-surprise/qchef_surveydata'
user_fam_scaled_arr, users_surp_ratings_arr = survey_reader(survey_fp)

Transpose for recipe level comparison

In [10]:
users_surp_ratings_userlvl = users_surp_ratings_arr
users_surp_ratings_recipelvl = list(np.transpose(users_surp_ratings_arr))

Flatten the arrays in both dimenssions

In [11]:
users_surp_ratings_user_flat = [item for sublist in users_surp_ratings_userlvl for item in sublist]
users_surp_ratings_recipe_flat = [item for sublist in list(np.transpose(users_surp_ratings_arr)) for item in sublist]

Mode: Oracle + Personalized, per user (default for oracle), 95th percentile

In [13]:
for user_idx, (each_personal_score, each_users_rating) in enumerate(zip(personalized_surp_estimates_95perc, users_surp_ratings_userlvl)):
	print user_idx, each_personal_score, each_users_rating
	print 'oracle_surp_estimates: spearmanr', spearmanr(oracle_surp_estimates_95perc, each_users_rating)
	print 'oracle_surp_estimates: pearsonr', pearsonr(oracle_surp_estimates_95perc, each_users_rating)
	print 'each_personal_score: spearmanr', spearmanr(each_personal_score, each_users_rating)
	print 'each_personal_score: pearsonr', pearsonr(each_personal_score, each_users_rating)

0 [3.4992518111744015, 1.820851219042163, 0.7769672304116093, 0.7493311274020704, 0.6586530676765014, 0.25178225605472326, 0.028929721010827643, -0.34728576461298233, -0.4468323932292995, -0.8863879953910269] [1, 1, 4, 4, 1, 4, 5, 4, 3, 4]
oracle_surp_estimates: spearmanr (0.30813875924572515, 0.38638381485150064)
oracle_surp_estimates: pearsonr (0.22543514207124077, 0.5311675107950075)
each_personal_score: spearmanr (-0.4785984984029348, 0.16171268487214444)
each_personal_score: pearsonr (-0.6849534673651423, 0.02884843147663722)
1 [3.8311755317516774, 2.5636580994075437, 1.49734122161952, 1.3938903613409888, 1.3311161275546537, 0.7757765059679845, 0.5436363806717961, 0.1339915324871328, 0.10003276519671964, -0.24937998554519694] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
oracle_surp_estimates: spearmanr (nan, nan)
oracle_surp_estimates: pearsonr (nan, 1.0)
each_personal_score: spearmanr (nan, nan)
each_personal_score: pearsonr (nan, 1.0)
2 [3.7035750118237103, 2.42695464967051, 1.269551777990076

Mode: Oracle + Personalized, per user (default for oracle), max score

In [14]:
for user_idx, (each_personal_score, each_users_rating) in enumerate(zip(personalized_surp_estimates_max, users_surp_ratings_userlvl)):
	print user_idx, each_personal_score, each_users_rating
	print 'oracle_surp_estimates: spearmanr', spearmanr(oracle_surp_estimates_max, each_users_rating)
	print 'oracle_surp_estimates: pearsonr', pearsonr(oracle_surp_estimates_max, each_users_rating)
	print 'each_personal_score: spearmanr', spearmanr(each_personal_score, each_users_rating)
	print 'each_personal_score: pearsonr', pearsonr(each_personal_score, each_users_rating)

0 [4.407839848709313, 2.353106981887523, 1.4716768665424258, 3.8059580332580722, 0.7603070766366502, 0.8105135465639222, 0.48790048028225025, 0.35901522166489186, -0.03543186751564214, -0.707503280261354] [1, 1, 4, 4, 1, 4, 5, 4, 3, 4]
oracle_surp_estimates: spearmanr (0.36058790975563576, 0.3060212830567241)
oracle_surp_estimates: pearsonr (0.2882286617927404, 0.4193198598657479)
each_personal_score: spearmanr (-0.3474756221281581, 0.3252140444837802)
each_personal_score: pearsonr (-0.42500349330409515, 0.22081690920088673)
1 [4.546031735017476, 2.6834683624340157, 2.1116708950495306, 4.57327267745409, 1.5656208905644358, 1.3540113489072998, 1.021739912141993, 0.4826758817105218, 1.0647990501959337, -0.04521917008970724] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
oracle_surp_estimates: spearmanr (nan, nan)
oracle_surp_estimates: pearsonr (nan, 1.0)
each_personal_score: spearmanr (nan, nan)
each_personal_score: pearsonr (nan, 1.0)
2 [4.363872028077286, 2.569457793870565, 4.4349466540026485, 2.0872

Mode: Personalized, per recipes, 95th percentile

In [15]:
for user_idx, (each_personal_score, each_users_rating) in enumerate(zip(personalized_surp_estimates_recipelvl_95perc, users_surp_ratings_recipelvl)):
	# print user_idx, each_personal_score, each_users_rating
	print 'each_personal_score: spearmanr', spearmanr(each_personal_score, each_users_rating)
	print 'each_personal_score: pearsonr', pearsonr(each_personal_score, each_users_rating)

each_personal_score: spearmanr (-0.07349350360068926, 0.5012731833778492)
each_personal_score: pearsonr (0.0470678194312292, 0.6669479823138214)
each_personal_score: spearmanr (-0.026899743770056325, 0.8057967170871672)
each_personal_score: pearsonr (0.021644582087360333, 0.8431940716727139)
each_personal_score: spearmanr (0.02565422493417843, 0.8146241887634227)
each_personal_score: pearsonr (0.08768868006866015, 0.4220671761363364)
each_personal_score: spearmanr (-0.07634221207085116, 0.48478369627816675)
each_personal_score: pearsonr (0.028425312054456007, 0.7950175575338468)
each_personal_score: spearmanr (-0.1735127088878394, 0.11011217670673425)
each_personal_score: pearsonr (-0.06869409584928696, 0.5297000176054651)
each_personal_score: spearmanr (0.06817197314333437, 0.5328403555052711)
each_personal_score: pearsonr (0.11039929267143753, 0.3115778256477339)
each_personal_score: spearmanr (-0.11270188427055053, 0.30153065942497115)
each_personal_score: pearsonr (-0.0073555879989

Mode: Personalized, per recipes, max score

In [16]:
for user_idx, (each_personal_score, each_users_rating) in enumerate(zip(personalized_surp_estimates_recipelvl_max, users_surp_ratings_recipelvl)):
	# print user_idx, each_personal_score, each_users_rating
	print 'each_personal_score: spearmanr', spearmanr(each_personal_score, each_users_rating)
	print 'each_personal_score: pearsonr', pearsonr(each_personal_score, each_users_rating)

each_personal_score: spearmanr (-0.029214263506742673, 0.7894579889520297)
each_personal_score: pearsonr (0.08728522340541112, 0.42421388253063297)
each_personal_score: spearmanr (-0.05088583914488773, 0.6417222038163688)
each_personal_score: pearsonr (-0.016086074364282218, 0.8831298846982172)
each_personal_score: spearmanr (0.038662552812219274, 0.7237680477987976)
each_personal_score: pearsonr (0.09837339908923808, 0.36752650926180475)
each_personal_score: spearmanr (-0.010636453959288496, 0.9225693406900467)
each_personal_score: pearsonr (0.02578326535562058, 0.8137085352877736)
each_personal_score: spearmanr (-0.15056567615228866, 0.16643219018854463)
each_personal_score: pearsonr (-0.11524720310984148, 0.2906736549563682)
each_personal_score: spearmanr (0.06469054877228342, 0.5540131601194482)
each_personal_score: pearsonr (0.10786901136275184, 0.3228655807664532)
each_personal_score: spearmanr (-0.12840358046461353, 0.23870653113446919)
each_personal_score: pearsonr (-0.02748620

Modes: Personalized, all users, user flat, 95th percentile/max score

In [17]:
print '95th percentile'
print 'spearmanr', spearmanr(personalized_surp_estimates_userflat_95perc, users_surp_ratings_user_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_userflat_95perc, users_surp_ratings_user_flat)
print 'Max score'
print 'spearmanr', spearmanr(personalized_surp_estimates_userflat_max, users_surp_ratings_user_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_userflat_max, users_surp_ratings_user_flat)

95th percentile
spearmanr (-0.30481904983015, 5.988071696087487e-20)
pearsonr (-0.31737659984687483, 1.4073000266857855e-21)
Max score
spearmanr (-0.27003791143270794, 7.773824208932607e-16)
pearsonr (-0.24738284583778478, 1.8555203868603025e-13)


Modes: Personalized, all recipes, recipe flat, 95th percentile/max score

In [18]:
print '95th percentile'
print 'spearmanr', spearmanr(personalized_surp_estimates_recipeflat_95perc, users_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_recipeflat_95perc, users_surp_ratings_recipe_flat)
print 'Max score'
print 'spearmanr', spearmanr(personalized_surp_estimates_recipeflat_max, users_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_recipeflat_max, users_surp_ratings_recipe_flat)

95th percentile
spearmanr (-0.30481904983015, 5.988071696087487e-20)
pearsonr (-0.31737659984687483, 1.4073000266857855e-21)
Max score
spearmanr (-0.27003791143270794, 7.773824208932607e-16)
pearsonr (-0.24738284583778472, 1.8555203868603025e-13)
